In [1]:
# important for gpuhub
# !pip install -r ../../requirements.txt --upgrade

In [2]:
import wandb
import sys
import os

import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

# load .env file
from dotenv import load_dotenv
from geo_model_trainer import GeoModelTrainer
from image_data_handler import ImageDataHandler

sys.path.insert(0, '../')
from data_loader import get_data_to_load, hash_filenames

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
WANDB_TOKEN = os.getenv('WANDB_TOKEN')
# Define where to run
env_path = '../../.env'
if not WANDB_TOKEN and os.path.exists(env_path):
  load_dotenv(env_path)
  WANDB_TOKEN = os.getenv('WANDB_TOKEN')

In [4]:
# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available.")
    
    # Print the name of the GPU
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    
    # Print the total and available memory
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9  # Convert bytes to GB
    print(f"Total Memory: {total_memory:.2f} GB")

    allocated_memory = torch.cuda.memory_allocated(0) / 1e9  # Convert bytes to GB
    print(f"Allocated Memory: {allocated_memory:.2f} GB")

    cached_memory = torch.cuda.memory_reserved(0) / 1e9  # Convert bytes to GB
    print(f"Cached Memory: {cached_memory:.2f} GB")

    # Print other properties
    device_properties = torch.cuda.get_device_properties(0)
    print(f"CUDA Capability: {device_properties.major}.{device_properties.minor}")
    print(f"Multi-Processor Count: {device_properties.multi_processor_count}")
else:
    print("No GPU found.")

No GPU found.


## Loading data

In [5]:
# set number of files to load
NUMBER_OF_FILES = 79000 # 100000
# Set to False to use non-mapped data (singleplayer distribution), has more data
USE_MAPPED = True

# get list with local data and file paths
list_files, zip_load_callback, additional_save_callback = get_data_to_load(loading_file='../3_data_preparation/04_data_cleaning/updated_data_list_more' if USE_MAPPED else '../3_data_preparation/04_data_cleaning/updated_data_list_non_mapped', 
                              file_location='../3_data_preparation/01_enriching/.data', image_file_location='../1_data_collection/.data', allow_new_file_creation=False, 
                              from_remote_only=True, download_link='default', limit=NUMBER_OF_FILES, shuffle_seed=42, allow_file_location_env=True, allow_json_file_location_env=True, 
                              allow_image_file_location_env=True, allow_download_link_env=True, return_zip_load_and_additional_save_callback=True)

All local files: 705681
Relevant files: 705681
Limited files: 158000


In [6]:
print(len(list_files) // 2)

79000


## Processing and loading data

In [7]:
data_augmentation = "full_augmentation"

# Default was 50, 50
image_size = [80, 130]
# Original size is  pixelHeight: 180, pixelWidth: 320
# image_size = [180, 320]

train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

preprocessing_config = { 'data_augmentation': data_augmentation, 'height': image_size[0], 'width': image_size[1], 'train_ratio': train_ratio, 'val_ratio': val_ratio, 'test_ratio': test_ratio }

base_transform = transforms.Compose([
          transforms.Resize((image_size[0], image_size[1])),
        ])
augmented_transform = None
final_transform = transforms.Compose([
          transforms.ToTensor(),
          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

if data_augmentation == "full_augmentation":
    augmented_transform = transforms.Compose([
        transforms.RandomPerspective(distortion_scale=0.75, p=0.5),  # Randomly apply perspective transformation
        transforms.RandomResizedCrop((image_size[0], image_size[1]), scale=(0.75, 1.0)),  # Randomly crop the image and resize it to the original size
        transforms.RandomRotation(10),          # Randomly rotate the image by up to 10 degrees
        transforms.ColorJitter(
            brightness=(0.5, 1.5),  # Randomly change brightness (lower limit to simulate night, upper limit for bright daylight)
            contrast=(0.5, 1.5),    # Randomly change contrast
            saturation=(0.5, 1.5),  # Randomly change saturation
            hue=(-0.1, 0.1)         # Randomly change hue
        )
    ])

In [8]:
# Creating Dataloasders with the classes

# Hash the files list to get a unique identifier for the data
hashed_filenames = hash_filenames(list_files)

cache = True

# Check if the code is running in a notebook
running_in_notebook = False
try:
  get_ipython()
  running_in_notebook = True
  print("Running in a notebook.")
except NameError:
  print("Running in a script.")

data_handler = ImageDataHandler(list_files, base_transform, augmented_transform, final_transform, preprocessing_config, batch_size=200, train_ratio=train_ratio, val_ratio=val_ratio, test_ratio=test_ratio, cache=cache, cache_zip_load_callback=zip_load_callback, cache_additional_save_callback=additional_save_callback, save_test_data=True, inspect_transformed=running_in_notebook and (data_augmentation == "full_augmentation"))
train_dataloader = data_handler.train_loader
val_dataloader = data_handler.val_loader
test_dataloader = data_handler.test_loader
country_to_index = data_handler.country_to_index
# Path of test data if it should be pushed to wandb
test_data_path = data_handler.test_data_path
# Previous run id if the test data was already pushed to wandb (to save space)
run_link = data_handler.run_link
# Path to the run link file to be created if it was not previously (if test data should be pushed)
run_link_path = data_handler.run_link_path

# Load the country_to_index mapping and print the count of different countries
print("Dataset size:", NUMBER_OF_FILES)
print("Dataset identifier:", hashed_filenames)
print(f"Count of different countries: {len(country_to_index)}")

Using cached data from: data_79000_data_augmentation=full_augmentationheight=80test_ratio=0.1train_ratio=0.7val_ratio=0.2width=130&3d999c168bedf247468b758771d441ecfd202c97836bed006c225a4afbc8688a.pth
Data loaded.
Using run link from: run_79000_data_augmentation=full_augmentationheight=80test_ratio=0.1train_ratio=0.7val_ratio=0.2width=130&3d999c168bedf247468b758771d441ecfd202c97836bed006c225a4afbc8688a.wandb
Skipping test data saving.
Dataset size: 79000
Dataset identifier: 3d999c168bedf247468b758771d441ecfd202c97836bed006c225a4afbc8688a
Count of different countries: 75


In [9]:
print("Number of train batches:", len(train_dataloader.dataset), "")

# Print first batch as an example, to see the structure
for images, coordinates, country_indices in train_dataloader:
    print("Images batch shape:", images.shape)
    print("Coordinates batch shape:", coordinates.shape)
    print(coordinates[0])
    print("Country indices:", country_indices.shape)
    print(country_indices[0])
    break

Number of train batches: 55300 
Images batch shape: torch.Size([200, 3, 80, 130])
Coordinates batch shape: torch.Size([200, 2])
tensor([-31.8840,  26.7955])
Country indices: torch.Size([200])
tensor(62)


## Training

In [10]:
model_types = ["efficientnet_b1", "mobilenet_v2", "resnet18", "efficientnet_b3", "resnet50"]
predict_coordinates=False
wandb.login(key=WANDB_TOKEN) if WANDB_TOKEN else wandb.login()

for model_type in model_types:
    if predict_coordinates:
        project_name = "predicting-coordinates"
        num_classes = 2
        sweep_goal = "minimize"
        sweep_metric_name = "Validation Distance (km)"
    else:
        num_classes = len(country_to_index)
        project_name = "predicting-country"
        sweep_goal = "maximize"
        sweep_metric_name = "Validation Accuracy Top 1"
    
    sweep_config = {
        "name": f"dspro2-basemodel-{model_type}-datasize-{NUMBER_OF_FILES}-input_imagesize-{image_size[0]}x{image_size[1]}",
        "method": "grid",
        "metric": {"goal": sweep_goal, "name": sweep_metric_name},
        "parameters": {
            "learning_rate": {"values": [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]},
            "optimizer": {"values": ["adamW"]},
            "weight_decay": {"values": [1e-1, 1e-2, 1e-3]},
            "epochs": {"values": [50]},
            "dataset_size": {"values": [NUMBER_OF_FILES]},
            "dataset_identifier": {"values": [hashed_filenames]},
            "seed": {"values": [42]},
            "model_name": {"values": [model_type]},
            "input_image_size": {"values": [image_size]},
            "predict_coordinates": {"values": [predict_coordinates]},
            "mapped_data": {"values": [USE_MAPPED]},
            "different_countries": {"values": [len(country_to_index)]},
            "data_augmentation": {"values": [data_augmentation]}
        },
    }
    
    sweep_id = wandb.sweep(sweep=sweep_config, project=f"dspro2-{project_name}")
    
    def set_run_link(config, run):
      global run_link
      global run_link_path
      if run_link_path is not None:
        run_link = run.id
        with open(run_link_path, 'w') as f:
          f.write(run_link)
        # Only write once
        run_link_path = None
        if additional_save_callback is not None:
          additional_save_callback()
      elif run_link is not None:
        wandb.log({"test_data_run_id": run_link})
    
    trainer = GeoModelTrainer(datasize=NUMBER_OF_FILES, train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                              num_classes=num_classes, predict_coordinates=predict_coordinates, country_to_index=country_to_index if not predict_coordinates else None, test_data_path=test_data_path, run_start_callback=set_run_link)

    wandb.agent(sweep_id, function=trainer.train)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: killusions. Use `wandb login --relogin` to force relogin


Create sweep with ID: ij6hrs1g
Sweep URL: https://wandb.ai/killusions/dspro2-predicting-country/sweeps/ij6hrs1g


wandb: Agent Starting Run: 6l9i2sjc with config:
wandb: 	data_augmentation: full_augmentation
wandb: 	dataset_identifier: 3d999c168bedf247468b758771d441ecfd202c97836bed006c225a4afbc8688a
wandb: 	dataset_size: 79000
wandb: 	different_countries: 75
wandb: 	epochs: 50
wandb: 	input_image_size: [80, 130]
wandb: 	learning_rate: 0.1
wandb: 	mapped_data: True
wandb: 	model_name: efficientnet_b1
wandb: 	optimizer: adamW
wandb: 	predict_coordinates: False
wandb: 	seed: 42
wandb: 	weight_decay: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Downloading: "https://download.pytorch.org/models/efficientnet_b1-c27df63c.pth" to /Users/linus/.cache/torch/hub/checkpoints/efficientnet_b1-c27df63c.pth
100%|██████████| 30.1M/30.1M [00:11<00:00, 2.83MB/s]
wandb: Ctrl + C detected. Stopping sweep.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: qv3wqfzm
Sweep URL: https://wandb.ai/killusions/dspro2-predicting-country/sweeps/qv3wqfzm
